In [5]:
import numpy as np
import matplotlib.pyplot as plt
import math
import timeit

import os
import sys
# root_path = os.getcwd() #+ "/scripts/personal/rick"
# sys.path.append(str(root_path))
sys.path.append("./")

# from quaos.core.prime_Functions_Andrew import *
# from quaos.core.prime_Functions_quditV2 import *
from quaos.symplectic import PauliSum, Pauli, PauliString
from quaos.gates import GateOperation, Circuit, Hadamard as H, SUM as CX, PHASE as S
from quaos.hamiltonian import random_pauli_hamiltonian, ground_state
np.set_printoptions(linewidth=200)

In [2]:
def xi(a, d):
    """
    Computes the a-th eigenvalue of a pauli with dimension d.
    
    Parameters:
        a (int): The integer to compute the eigenvalue for.
        d (int): The dimension of the pauli to use.
    
    Returns:
        complex: The computed eigenvalue.
    """
    return np.exp(2 * np.pi * 1j * a / d)

def reduce_qudits(P):
    C,pivots = symplectic_RREF_qudit(P)
    return C,pivots

def symplectic_RREF_qudit(P):
    d = P.dimensions
    P1 = P.copy()
    C = Circuit(d)
    pivots = []

    for i in range(P.n_qudits()):
        C,pivots = symplectic_RREF_iter_qudit_(P1.copy(),C,pivots,i)
    P1 = C.act(P1)

    removable_qubits = set(range(q))-set([pivot[1] for pivot in pivots])
    conditional_qubits = sorted(set(range(q))-removable_qubits-set([pivot[1] for pivot in pivots if pivot[2]=='Z']))
    if any(conditional_qubits):
        g = H(conditional_qubits, d)#gate(H,conditional_qubits)
        C.add_gates_(g)
        P1 = g.act(P1)

    return C,sorted(pivots,key=lambda x:x[1])

def number_of_SUM_X(r_control,r_target,d):
    N = 1
    while (r_target + N*r_control)%d != 0:
        if N > d:
            raise Exception('Error in Exponents r_control = '+str(r_control)+' r_target = '+ str(r_target))
        N += 1
        
    return(N)

def number_of_SUM_Z(s_control,s_target,d):
    N = 1
    while (s_control - N*s_target)%d != 0:
        if N > d:
            raise Exception('Error in Exponents s_control = '+str(s_control)+' s_target = '+ str(s_target))
        N += 1
        
    return(N)

def number_of_S(x_exp,z_exp,d):
    N = 1
    while (x_exp*N + z_exp)%d != 0:
        if N > d:
            raise Exception('Error in Exponents x_exp = '+str(x_exp)+' z_exp = '+ str(z_exp))
        N += 1
        
        
    return(N)

def cancel_X(pauli_sum,qudit,pauli_index,C,q_max):
    list_of_gates = []
    for i in range(qudit+1,q_max):
        if pauli_sum.x_exp[pauli_index,i]:
            list_of_gates += [CX(qudit, i, pauli_sum.dimensions[qudit])]*number_of_SUM_X(pauli_sum.x_exp[pauli_index,qudit],pauli_sum.x_exp[pauli_index,i],pauli_sum.dimensions[i])
    C.add_gate(list_of_gates)     
    for g in list_of_gates:
        pauli_sum = g.act(pauli_sum)
    return(pauli_sum,C)

def cancel_Z(pauli_sum,qudit,pauli_index,C,q_max):
    list_of_gates = []
    list_of_gates += [H(qudit, pauli_sum.dimensions[qudit])]
    for i in range(qudit+1,q_max):
        if pauli_sum.z_exp[pauli_index,i]:
            list_of_gates += [CX(i, qudit, pauli_sum.dimensions[qudit])]*number_of_SUM_Z(pauli_sum.z_exp[pauli_index,i],pauli_sum.x_exp[pauli_index,qudit],pauli_sum.dimensions[i])
    list_of_gates += [H(qudit, pauli_sum.dimensions[qudit])]
    C.add_gate(list_of_gates)     
    for g in list_of_gates:
        pauli_sum = g.act(pauli_sum)
    return(pauli_sum,C)

def cancel_Y(pauli_sum,qudit,pauli_index,C):
    list_of_gates = [S(qudit, pauli_sum.dimensions[qudit])]*number_of_S(pauli_sum.x_exp[pauli_index,qudit],
                                                                       pauli_sum.z_exp[pauli_index,qudit],
                                                                       pauli_sum.dimensions[qudit])
    C.add_gates_(list_of_gates)
    for g in list_of_gates:
        pauli_sum = g.act(pauli_sum)
    return(pauli_sum,C)

def cancel_pauli(pauli_sum, qudit, pauli_index, circuit, n_q_max):
    if any(pauli_sum.x_exp[pauli_index,i] for i in range(qudit+1,n_q_max)):
        pauli_sum,circuit = cancel_X(pauli_sum,qudit,pauli_index,circuit,n_q_max)

    if any(pauli_sum.z_exp[pauli_index,i] for i in range(qudit+1,n_q_max)):
        pauli_sum,circuit = cancel_Z(pauli_sum,qudit,pauli_index,circuit,n_q_max)

    # if Pauli a1, qubit a is Y, add S gate to make it X
    if pauli_sum.z_exp[pauli_index,qudit]:
        pauli_sum,circuit = cancel_Y(pauli_sum,qudit,pauli_index,circuit)

    return pauli_sum,circuit


def symplectic_RREF_iter_qudit_(P,C,pivots,current_qudit):
    n_p, n_q = P.n_paulis(),P.n_qudits()
    P = C.act(P)
    n_q_max = n_q
    for i in range(n_q-current_qudit):
        if P.dimensions[current_qudit+i] != P.dimensions[current_qudit]:
            n_q_max = current_qudit + i - 1
            break

    if any(P.x_exp[:,current_qudit]) or any(P.z_exp[:,current_qudit]):
        if not any(P.x_exp[:,current_qudit]):
            g = H(current_qudit,P.dimensions[current_qudit])
            C.add_gate(g)
            P = g.act(P)

        current_pauli = min(i for i in range(n_p) if P.x_exp[i,current_qudit])  # first Pauli that has an x-component
        pivots.append((current_pauli,current_qudit,'X'))

        P, C = cancel_pauli(P, current_qudit, current_pauli, C, n_q_max)

    if any(P.z_exp[:,current_qudit]):
        current_pauli = min(i for i in range(n_p) if P.z_exp[i, current_qudit])
        pivots.append((current_pauli,current_qudit,'Z'))

        g = H(current_qudit,P.dimensions[current_qudit])
        C.add_gate(g)
        P = g.act(P)

        P, C = cancel_pauli(P, current_qudit, current_pauli, C, n_q_max)

        g = H(current_qudit,P.dimensions[current_qudit])
        C.add_gate(g)
        P = g.act(P)
    
    return C,pivots

def reduced_qudit_Hamiltonian(P,cc):
    PP1, ccc1 = [], []
    p,q = P.n_paulis(),P.n_qudits()
    C,pivots = reduce_qudits(P)
    P1 = C.act(P)
    print(P1)

    removable_qudits = set(range(q))-set([pivot[1] for pivot in pivots])
    conditional_qudits = set(range(q))-removable_qudits-set([pivot[1] for pivot in pivots if pivot[2]=='Z'])
    removable_qudits = sorted(removable_qudits)
    conditional_qudits = sorted(conditional_qudits)
    
    conditional_factors = []
    for i0 in range(p):
        conditional_factors.append([P1.z_exp[i0,i1] for i1 in conditional_qudits])

    P2 = P1.copy()
    P2.delete_qudits_(set(removable_qudits)|set(conditional_qudits))
    print("Q",q)
    print("R",len(removable_qudits))
    print("C",len(conditional_qudits))
    print("Removed",removable_qudits)
    print("Conditioned",conditional_qudits)
    if len(conditional_qudits) > 0:
        Pz_ss = []
        for i in conditional_qudits:
            for k in range(1,P.dimensions[i]):
                ss = ""
                for j in range(q):
                    ss += "x0"
                    if i == j:
                        ss += "z"+str(k)
                    else:
                        ss += "z0"
                    ss += " "
                ss = ss[:-1]
                Pz_ss.append(ss)

        Pz = PauliSum(Pz_ss, dimensions=P.dimensions)#string_to_pauli(Pz_ss,dims=P.dims)
        Pz = act_inverse(Pz,C)
        print("Pauli Symmetries:")
        Pz.print()

    for conditions in itertools.product(*[np.arange(P1.dims[conditional_qudits[i]]) for i in range(len(conditional_qudits))],repeat=1):
        cc1 = [cc[i0] * xi(np.sum(np.array(conditions) * np.array(conditional_factors[i0]) * P2.lcm/np.array(P1.dims[conditional_qudits])),P2.lcm) for i0 in range(p)]
        #cc1 = [cc[i0] * xi(np.sum(np.array(conditions) * np.array(conditional_factors[i0]))%P2.lcm, P2.lcm) for i0 in range(p)]
        cc2 = np.copy(cc1)
        P3 = P2.copy()
        P2_ss = pauli_to_string(P2)[0]
        
        # find duplicates
        duplicates = []
        for i in range(P2.paulis()):
            for j in range(i+1,P2.paulis()):
                if P2_ss[i] == P2_ss[j]:
                    already_a_duplicate = False
                    for k,dd in enumerate(duplicates):
                        if i in dd or j in dd:
                            duplicates[k] = dd | {i,j}
                            already_a_duplicate = True
                    if already_a_duplicate == False:
                        duplicates.append({i,j})
        
        # unify duplicates
        excess_paulis = []
        for dd in duplicates:
            i = sorted(list(dd))[0]
            cc2[i] = np.sum([cc1[j]*xi(P2.phases[j],P2.lcm) for j in dd])
            P3.phases[i] = 0
            excess_paulis += [j for j in sorted(list(dd))[1:]]
        P3.delete_paulis_(excess_paulis)
        cc3 = np.delete(cc2,excess_paulis)
        
        # add a part to remove paulis with coeficients that are 0
        zero_paulis = []
        for i,c in enumerate(cc3):
            if np.abs(c) < 10**(-10):
                zero_paulis.append(i)
        P3.delete_paulis_(zero_paulis)
        cc3 = np.delete(cc3,zero_paulis)
        PP1.append(P3)
        ccc1.append(cc3)
    
    return PP1,ccc1,C
    

In [3]:
P = random_pauli_hamiltonian(2, [3, 3],mode='one')
# print(P)

psi = ground_state(P)

x0z0 x2z1 x2z2 x1z1 x2z1 x1z1 x1z2
x0z0 x1z2 x1z1 x2z2 x1z2 x2z2 x2z1
x1z2 x1z2 x1z1 x1z2 x0z2 x2z1 x1z1
x2z1 x2z1 x2z2 x2z1 x0z1 x1z2 x2z2
x0z1 x0z1 x0z1 x0z2 x2z0 x1z0 x0z1
x0z2 x0z2 x0z2 x0z1 x1z0 x2z0 x0z2
x1z0 x2z2 x0z1 x0z0 x2z2 x0z1 x1z1
x2z0 x1z1 x0z2 x0z0 x1z1 x0z2 x2z2
x2z1 x0z1 x0z1 x1z2 x2z1 x1z2 x2z2
x1z2 x0z2 x0z2 x2z1 x1z2 x2z1 x1z1
x2z1 x0z0 x2z1 x0z0 x0z2 x0z2 x0z1
x1z2 x0z0 x1z2 x0z0 x0z1 x0z1 x0z2
3333333 3333333
------- -------
0221211 0121112 0/3
0112122 0212221 0/3
1111021 2212211 0/3
2222012 1121122 0/3
0000210 1112001 0/3
0000120 2221002 0/3
1200201 0210211 0/3
2100102 0120122 0/3
2001212 1112122 0/3
1002121 2221211 0/3
2020000 1010221 0/3
1010000 2020112 0/3
GSE: (-5.154141398385708+1.3426559482628695e-14j)


In [4]:
circuit,pivots = reduce_qudits(P)
pauli_sum = circuit.act(P)
print(pauli_sum)
psi1 = ground_state(pauli_sum,cc)
print("GSE:",Hamiltonian_Mean(pauli_sum,cc,psi1))

x2z0 x1z0 x0z0 x0z0 x0z0 x0z0 x0z0
x1z0 x2z0 x0z0 x0z0 x0z0 x0z0 x0z0
x2z0 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0
x1z0 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0
x0z2 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0
x0z1 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0
x1z2 x1z2 x0z0 x0z0 x0z0 x0z0 x0z0
x2z1 x2z1 x0z0 x0z0 x0z0 x0z0 x0z0
x1z2 x0z0 x0z1 x0z0 x0z0 x0z0 x0z0
x2z1 x0z0 x0z2 x0z0 x0z0 x0z0 x0z0
x2z2 x2z1 x1z0 x0z0 x0z0 x0z0 x0z0
x1z1 x1z2 x2z0 x0z0 x0z0 x0z0 x0z0

GSE: (-5.154141398385709+7.283036292456834e-15j)


In [ ]:
PP1,ccc1,circuit = reduced_qudit_Hamiltonian(P,cc)
print()
print('Reduced Hamiltonians')
for i,pauli_sum in enumerate(PP1):
    print('Hamiltonian Nr.',i)
    pauli_sum.print()
    print()
    psi1 = ground_state(pauli_sum,ccc1[i])
    print("GSE:",Hamiltonian_Mean(pauli_sum,ccc1[i],psi1))
    print()

x2z0 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0
x1z0 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0
x2z1 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0
x1z2 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0
x0z0 x2z0 x2z0 x0z0 x0z0 x0z0 x0z0
x0z0 x1z0 x1z0 x0z0 x0z0 x0z0 x0z0
x2z0 x2z0 x0z0 x0z0 x0z0 x0z0 x0z0
x1z0 x1z0 x0z0 x0z0 x0z0 x0z0 x0z0
x0z2 x0z2 x0z0 x0z0 x0z0 x0z0 x0z0
x0z1 x0z1 x0z0 x0z0 x0z0 x0z0 x0z0
x2z1 x2z2 x1z1 x0z0 x0z0 x0z0 x0z0
x1z2 x1z1 x2z2 x0z0 x0z0 x0z0 x0z0
Q 7
R 4
C 0
Removed [3, 4, 5, 6]
Conditioned []

Reduced Hamiltonians
Hamiltonian Nr. 0
x2z0 x0z0 x0z0
x1z0 x0z0 x0z0
x2z1 x0z0 x0z0
x1z2 x0z0 x0z0
x0z0 x2z0 x2z0
x0z0 x1z0 x1z0
x2z0 x2z0 x0z0
x1z0 x1z0 x0z0
x0z2 x0z2 x0z0
x0z1 x0z1 x0z0
x2z1 x2z2 x1z1
x1z2 x1z1 x2z2

GSE: (-5.262822164281169+6.283262633502884e-15j)

